# Finding and Replacing Missing Data

In [1]:
import pandas as pd
import numpy as np
import os 

## Load and Inspect the Data

In [2]:
filename = os.path.join(os.getcwd(), "data", "adult.data.partial.missing")
df = pd.read_csv(filename, header=0)

In [3]:
df.shape

(7000, 15)

In [4]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,36.0,State-gov,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45.0,United-States,<=50K
1,35.0,Private,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45.0,United-States,<=50K
2,21.0,Private,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16.0,United-States,<=50K
3,45.0,Private,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24.0,Canada,>50K
4,39.0,Private,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40.0,United-States,>50K



## Dealing with Missing Data

Our goal will be to identify which columns in a dataset have missing values, and to replace a missing value in a column with the mean of the other values in that column. We will add dummy variables to our dataset to indicate which  columns initally had missing values. 

### Step 1:  Identify Missing Values Using Pandas `isnull()` Method

First let us check if there are missing values in DataFrame `df`.

In [5]:
df.isnull().values.any()

True

DataFrame `df` contains missing values! The Pandas `isnull()` method returns `True`/`False` values indicating whether a value is or is not missing in a particular position in a DataFrame or Series. This method recognizes various spellings of missingness like `NaN`, `nan`, `None`, and `NA` among others.<br> Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html) for more information.

In [6]:
df.isnull().head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


The code cell below counts the number of times a missing value occurs in each column. It applies the `isnull()` method and then aggregates the results by columns using the `np.sum()` function. For more information about `np.sum()`, consult the online [documentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

In [7]:
nan_count = np.sum(df.isnull(), axis = 0)
nan_count

age                35
workclass         375
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        375
relationship        0
race                0
sex_selfID          0
capital-gain        0
capital-loss        0
hours-per-week     70
native-country    138
label               0
dtype: int64

The code cell below stores the names of the columns with detected missing values into a Python list.

In [8]:
condition = nan_count != 0 # look for all columns with missing values

col_names = nan_count[condition].index # get the column names
print(col_names)

nan_cols = list(col_names) # convert column names to list
print(nan_cols)

Index(['age', 'workclass', 'occupation', 'hours-per-week', 'native-country'], dtype='object')
['age', 'workclass', 'occupation', 'hours-per-week', 'native-country']


### Step 2: Choose Which Values to Fill 

We can see that five columns in our DataFrame contain missing values. Would you want to replace the missing values with something for every one of these columns? 

Let's take a look at the data types of the columns that contain missing values using `dtypes`.

In [9]:
nan_col_types = df[nan_cols].dtypes
nan_col_types

age               float64
workclass          object
occupation         object
hours-per-week    float64
native-country     object
dtype: object

For three of the five identified columns, the type is 'object'. Is this a problem?<br>
    A common approach to dealing with the missing values is to replace those values with either the mean, the median, or some other type of 'representative' value wherever a `nan` occurs. This, of course, assumes that the column is numerical to begin with. That doesn't seem to be true for the `workclass`, `occupation`, and `native-country` variables.
Let us confirm:

In [10]:
print(df['workclass'].unique())
print(df['occupation'].unique())
print(df['native-country'].unique())

['State-gov' 'Private' nan 'Self-emp-not-inc' 'Local-gov' 'Self-emp-inc'
 'Federal-gov' 'Without-pay']
['Prof-specialty' 'Handlers-cleaners' 'Other-service' 'Adm-clerical'
 'Craft-repair' 'Sales' nan 'Exec-managerial' 'Farming-fishing'
 'Machine-op-inspct' 'Transport-moving' 'Tech-support' 'Priv-house-serv'
 'Protective-serv' 'Armed-Forces']
['United-States' 'Canada' 'England' 'Germany' 'Cuba' nan 'Puerto-Rico'
 'Mexico' 'Nicaragua' 'China' 'South' 'India' 'Vietnam' 'Philippines'
 'El-Salvador' 'Guatemala' 'Japan' 'Jamaica' 'Peru' 'France' 'Greece'
 'Italy' 'Columbia' 'Honduras' 'Iran' 'Poland' 'Haiti'
 'Dominican-Republic' 'Scotland' 'Yugoslavia' 'Trinadad&Tobago' 'Ireland'
 'Portugal' 'Taiwan' 'Hong' 'Ecuador' 'Laos' 'Hungary' 'Thailand'
 'Outlying-US(Guam-USVI-etc)' 'Cambodia']


The concept of 'mean' is not defined for string entries, so filling in the missing values with the mean of the column wouldn't work here. In real business settings, one way to go about filling in the missing values would be to fit a model that predicts the country based on other values. All data filling methods come with caveats, and some may threaten the validity of your larger analytical conclusions.

For the rest of this exercise, we will focus only on the numerical variables, for which it makes sense to replace every missing value with mean of the column. Those are `age` and `hours-per-week` columns.

###  Step 3: Create 'Dummy' Variables for Missing Values

No method of imputing missing values is perfect, and for this reason it makes sense to keep track of which values we artificially created. 

The code cell below looks at the the values in columns `age` and `hours-per-week` and stores the corresponding `True`/`False` values (True if the value is missing and False if the value is present) in new columns `age_na` and `hours-per-week_na`. Run the cell and inspect the new columns.

In [11]:
df['age_na'] = df['age'].isnull()
df['hours-per-week_na'] = df['hours-per-week'].isnull()
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label,age_na,hours-per-week_na
0,36.0,State-gov,112074,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Non-Female,0,0,45.0,United-States,<=50K,False,False
1,35.0,Private,32528,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Non-Female,0,0,45.0,United-States,<=50K,False,False
2,21.0,Private,270043,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,16.0,United-States,<=50K,False,False
3,45.0,Private,168837,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,24.0,Canada,>50K,False,False
4,39.0,Private,297449,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,0,0,40.0,United-States,>50K,False,False


### Step 4: Fill the Missing Values Using Pandas `fillna()` Method

The Pandas `fillna()` method is used to "fill in" missing values in a Series or DataFrame object. Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) for more information about how to use the `fillna()` method.
The code cell below uses `fillna()` to fill in values for the missing values in the `age` column.
It fills in the missing values with the mean value of all of the existing values in the that column. It uses the Pandas `mean()` method to compute the replacement values. 
For more information about `mean()`, consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.mean.html).

Tip: when working with `fillna()`, make sure that you do not just create a copy object with the filled values, but change the original values of the `df` object by specifying the `inplace = True` parameter value.

First inspect some of the columns that contain missing values.

In [12]:
df.loc[df['age'].isnull()]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex_selfID,capital-gain,capital-loss,hours-per-week,native-country,label,age_na,hours-per-week_na
453,NaN,Private,117166,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Non-Female,0,0,50.0,United-States,<=50K,True,False
654,NaN,NaN,65545,Masters,14,Divorced,NaN,Own-child,White,Female,0,0,55.0,United-States,<=50K,True,False
865,NaN,Self-emp-not-inc,93806,Some-college,10,Married-civ-spouse,Sales,Husband,White,Non-Female,0,0,55.0,United-States,<=50K,True,False
1206,NaN,Private,441637,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Non-Female,0,0,40.0,United-States,<=50K,True,False
1262,NaN,Private,350440,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Non-Female,0,0,40.0,United-States,>50K,True,False
1302,NaN,Private,317443,Some-college,10,Never-married,Adm-clerical,Own-child,Black,Female,0,0,15.0,United-States,<=50K,True,False
1496,NaN,Private,99185,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Non-Female,7298,0,50.0,United-States,>50K,True,False
2100,NaN,Private,179271,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Non-Female,0,0,50.0,United-States,>50K,True,False
2581,NaN,Private,145160,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Non-Female,0,0,43.0,United-States,<=50K,True,False
2651,NaN,Private,151580,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Non-Female,15024,0,40.0,United-States,>50K,True,False


In [13]:
# look at one row that contains a missing value for age
print("Row 654:  " + str(df['age'][654]))

# compute mean for all non null age values
mean_ages=df['age'].mean()
print("mean value for all age columns: " + str(mean_ages))

# fill all missing values with the mean
df['age'].fillna(value=mean_ages, inplace=True)

# look at one of the rows that contained a missing value for age. 
# It should now contain the mean
print("Row 654:  " + str(df['age'][654]))



Row 654:  nan
mean value for all age columns: 38.61981335247667
Row 654:  38.61981335247667


In the code cell below, do the same for the `hours-per-week` column.

1. Compute the mean value of the `hours-per-week` column and save the result to variable `mean_hours`
2. Use `fillna` to change the values of the missing columns to `mean_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [16]:
mean_hours=df['hours-per-week'].mean()
df['hours-per-week'].fillna(value=mean_hours, inplace=True)


### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [17]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testFillNa

try:
    p, err = testFillNa(df)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!


Check if we successfully converted all missing values to the mean value. Display the sum of missing values for the `age` column. 

In [18]:
np.sum(df['age'].isnull(), axis = 0)

0

In the code cell below, do the same for the `hours-per-week` column. Save the result to variable `sum_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [19]:
sum_hours = np.sum(df['hours-per-week'].isnull(), axis = 0)

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [20]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testSumHours

try:
    p, err = testSumHours(df, sum_hours)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Correct!
